In [36]:
from datetime import datetime, timedelta

class WinVaultTokenomics:
    def __init__(self, staking_token, rewards_token, duration):
        self.staking_token = staking_token
        self.rewards_token = rewards_token
        self.owner = 'owner_address'
        self.duration = duration
        self.finish_at = None
        self.updated_at = None
        self.reward_rate = 0
        self.reward_per_token_stored = 0
        self.total_supply = 0
        self.balance_of = {}
        self.user_reward_per_token_paid = {}
        self.rewards = {}

    def only_owner(func):
        def wrapper(self, *args, **kwargs):
            if kwargs.get('caller') != self.owner:
                raise PermissionError("not authorized")
            return func(self, *args, **kwargs)
        return wrapper

    def update_reward(self, account):
        self.reward_per_token_stored = self.reward_per_token()
        self.updated_at = self.last_time_reward_applicable()
        if account:
            self.rewards[account] = self.earned(account)
            self.user_reward_per_token_paid[account] = self.reward_per_token_stored

    def last_time_reward_applicable(self):
        if self.finish_at:
            return min(self.finish_at, datetime.now())
        return datetime.now()

    def reward_per_token(self):
        if self.total_supply == 0:
            return self.reward_per_token_stored

        delta_time = (self.last_time_reward_applicable() - self.updated_at).total_seconds()
        return self.reward_per_token_stored + (self.reward_rate * delta_time * 1e18) / self.total_supply

    def stake(self, user, amount):
        self.update_reward(user)
        if amount <= 0:
            raise ValueError("amount = 0")
        self.staking_token.transfer_from(user, 'contract_address', amount)
        if user not in self.balance_of:
            self.balance_of[user] = 0
        self.balance_of[user] += amount
        self.total_supply += amount

    def withdraw(self, user, amount):
        self.update_reward(user)
        if amount <= 0:
            raise ValueError("amount = 0")
        self.balance_of[user] -= amount
        self.total_supply -= amount
        self.staking_token.transfer(user, amount)

    def earned(self, account):
        return (self.balance_of.get(account, 0) * (self.reward_per_token() - self.user_reward_per_token_paid.get(account, 0))) / 1e18 + self.rewards.get(account, 0)

    def get_reward(self, user):
        self.update_reward(user)
        reward = self.rewards.get(user, 0)
        if reward > 0:
            self.rewards[user] = 0
            self.rewards_token.transfer(user, reward)

    @only_owner
    def set_rewards_duration(self, duration, caller=None):
        if self.finish_at and self.finish_at > datetime.now():
            raise ValueError("reward duration not finished")
        self.duration = duration

    @only_owner
    def notify_reward_amount(self, amount, caller=None):
        self.update_reward(None)
        if not self.finish_at or datetime.now() >= self.finish_at:
            self.reward_rate = amount / self.duration.total_seconds()
        else:
            remaining_rewards = (self.finish_at - datetime.now()).total_seconds() * self.reward_rate
            self.reward_rate = (amount + remaining_rewards) / self.duration.total_seconds()

        if self.reward_rate <= 0:
            raise ValueError("reward rate = 0")
        if self.reward_rate * self.duration.total_seconds() > self.rewards_token.balance_of('contract_address'):
            raise ValueError("reward amount > balance")

        self.finish_at = datetime.now() + self.duration
        self.updated_at = datetime.now()

class Token:
    def __init__(self, name, total_supply):
        self.name = name
        self.total_supply = total_supply
        self.balances = {}

    def balance_of(self, account):
        return self.balances.get(account, 0)

    def transfer(self, recipient, amount):
        if self.balances.get('contract_address', 0) < amount:
            raise ValueError("Insufficient balance")
        self.balances['contract_address'] -= amount
        if recipient not in self.balances:
            self.balances[recipient] = 0
        self.balances[recipient] += amount

    def transfer_from(self, sender, recipient, amount):
        if self.balances.get(sender, 0) < amount:
            raise ValueError("Insufficient balance")
        self.balances[sender] -= amount
        if recipient not in self.balances:
            self.balances[recipient] = 0
        self.balances[recipient] += amount

# Example usage
staking_token = Token("StakingToken", 1_000_000)
rewards_token = Token("RewardsToken", 1_000_000)
win_vault = WinVaultTokenomics(staking_token, rewards_token, timedelta(days=30))

# Initialize balances for the example
staking_token.balances['user1'] = 10_000
staking_token.balances['user2'] = 15_000
staking_token.balances['user3'] = 25_000
rewards_token.balances['contract_address'] = 50_000

win_vault.stake('user1', 5_000)
win_vault.stake('user2', 15_000)
win_vault.stake('user3', 25_000)
win_vault.notify_reward_amount(10_000, caller='owner_address')

# Simulate time passing for rewards calculation
win_vault.updated_at -= timedelta(days=30)

rewards = {
    user: win_vault.earned(user) for user in ['user1', 'user2', 'user3']
}

for user, reward in rewards.items():
    print(f"{user} earns {reward:.2f} reward tokens")

win_vault.get_reward('user1')
win_vault.get_reward('user2')
win_vault.get_reward('user3')

for user in ['user1', 'user2', 'user3']:
    print(f"{user} new balance: {rewards_token.balance_of(user)}")


user1 earns 1111.11 reward tokens
user2 earns 3333.33 reward tokens
user3 earns 5555.56 reward tokens
user1 new balance: 1111.111111244427
user2 new balance: 3333.3333337705753
user3 new balance: 5555.55555632716
